# My first Deep Learning Algorithm

### Import the relevant packages

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

### Loading the data

In [2]:
image_data_set, mnist_image_info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)

### Extracting training and testing data

In [3]:
training_data, testing_data = image_data_set['train'], image_data_set['test']

# extracting our validation samples from the training samples (as a percentage of the training samples)

number_of_validation_samples = 0.1 * mnist_image_info.splits['train'].num_examples

integer_validation_samples = tf.cast(number_of_validation_samples, tf.int64)

In [4]:
# converting number of test samples to integer (as float would give an error)

number_of_test_samples = mnist_image_info.splits['test'].num_examples


integer_test_samples = tf.cast(number_of_test_samples, tf.int64)

In [5]:
# # scaling our input data

# we'll create a function for that...
def scaling_algorithm(input_image, label):
    output_image = tf.cast(input_image, tf.float32)

    # dividing each input by 255 to scale them between 0 and 1
    output_image /= 255

    # releasing our output...
    return output_image, label

scaled_train_and_validation_dataset = training_data.map(scaling_algorithm)

scaled_test_dataset = testing_data.map(scaling_algorithm)

In [6]:
# we need to shoffle our data, to make it completely random, because it is possible that the computer arranged them in an ascending order, which is very dangerous for our SGD optimizer, and will make it confused
buffer_size_for_shuffling = 12985  # i don't like giving it definite numbers like 10000 or 12000. i like to make it random

shuffled_TandV_dataset = scaled_train_and_validation_dataset.shuffle(buffer_size_for_shuffling)

In [7]:
# now, we need to extract our validation samples from the entire training and validation dataset

validation_dataset = shuffled_TandV_dataset.take(integer_validation_samples) # note that integer_validation_samples is a number

training_dataset = shuffled_TandV_dataset.skip(integer_validation_samples) # our training dataset becomes everything that's not on the validation dataset

In [8]:
# we'll divide our training data into batches

size_per_batch = 100  # 100 samples per batch

training_dataset = training_dataset.batch(size_per_batch) # the training data is being splitted into batches based on the specified number of samples per batch

validation_dataset = validation_dataset.batch(integer_validation_samples)  # the validation data only has one batch, which is the number of samples in the validation data, this is because the validation dataset is only going to be used for forward propagation, as our model is using it to check whether it's doing well or not. it won't encounter back propagation, because the model won't train on it

testing_dataset = scaled_test_dataset.batch(integer_test_samples) # the validation data only has one batch, which is the number of samples in the validation data, this is because the validation dataset is only going to be used for forward propagation, as our model is using it to test it's performance. it won't encounter back propagation, because the model won't train on it


# now we will iterate the validation data

validation_input_group, validation_targets_group = next(iter(validation_dataset))

## Building our model

<p>There'll be 4 layers:<br>An input layer<br>2 hidden layers<br>An output layer</p>
<p>You can adjust them later if you want</p>

In [9]:
model_input_size = 784  # 28 x 28 pixlels all stacked into one vector which is 784

# our model output will be a number from 0 to 9
model_output_size = 10

hidden_layers_neuron_size = 213

# each image has a dimension of 28 by 28 by 1
tensor_input_shape = (28, 28, 1)

activation_functions = ['relu', 'softmax', 'sigmoid', 'tanh', 'leaky_relu']

# defining our model structure...
image_recognition_algorithm = tf.keras.Sequential([
    # we need to flatten our input tensor to one vector, thus we will use the "Flatten" method

    tf.keras.layers.Flatten(input_shape = tensor_input_shape), # this is our input that has been flattened

    # now, we'll use the "Dense" method that will compute the dot product of the inputs and weights and add the biases

    # output = (inputs * weights) + biases
    tf.keras.layers.Dense(hidden_layers_neuron_size, activation = activation_functions[0]), # this is our first hidden layer

    tf.keras.layers.Dense(hidden_layers_neuron_size, activation = activation_functions[3]), # this is our second hidden layer

    # third layer...
    tf.keras.layers.Dense(hidden_layers_neuron_size, activation = activation_functions[2]),

    # now, unto our output layer...
    # we'll use "softmax" as the activation function for our output layer
    tf.keras.layers.Dense(model_output_size, activation = activation_functions[1])
])

c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### Selecting our optimizer and loss function

In [10]:
custom_optimizers = [tf.keras.optimizers.Adagrad(learning_rate = 0.0001), tf.keras.optimizers.Adagrad(learning_rate = 0.02), 'adam']

image_recognition_algorithm.compile(optimizer = custom_optimizers[2], loss = 'sparse_categorical_crossentropy', metrics = ['accuracy']) # here, we can use different optimizers, "sparse_categorical_crossentropy" as our loss function, and "accuracy" as one of our models parameters we want to view

### Training our model

In [11]:
# now, let's train the model we've worked hard to build...
# we'll specify the number of epochs we want our model to go through

specified_number_of_epochs = 6

image_recognition_algorithm.fit(training_dataset, epochs = specified_number_of_epochs, validation_data = (validation_input_group, validation_targets_group), verbose = 2)

Epoch 1/6
540/540 - 10s - 19ms/step - accuracy: 0.9124 - loss: 0.3058 - val_accuracy: 0.9602 - val_loss: 0.1293
Epoch 2/6
540/540 - 5s - 9ms/step - accuracy: 0.9692 - loss: 0.1032 - val_accuracy: 0.9732 - val_loss: 0.0877
Epoch 3/6
540/540 - 5s - 9ms/step - accuracy: 0.9793 - loss: 0.0683 - val_accuracy: 0.9818 - val_loss: 0.0561
Epoch 4/6
540/540 - 4s - 8ms/step - accuracy: 0.9849 - loss: 0.0493 - val_accuracy: 0.9862 - val_loss: 0.0435
Epoch 5/6
540/540 - 5s - 9ms/step - accuracy: 0.9874 - loss: 0.0399 - val_accuracy: 0.9875 - val_loss: 0.0387
Epoch 6/6
540/540 - 5s - 9ms/step - accuracy: 0.9904 - loss: 0.0307 - val_accuracy: 0.9840 - val_loss: 0.0466
